In [ ]:
!pip install -q sentence_transformers

!pip install -q bert-tensorflow


In [24]:
# Imports
import os
import re
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from bert.tokenization import FullTokenizer
from tensorflow.keras import backend as K
import numpy as np
from tqdm import tqdm
import email

from sentence_transformers import SentenceTransformer, util


In [121]:
df = pd.read_csv("C:/cursuri/Facultate/Master_An2/SRI/emails.csv")
print("Loaded {} emails with {} columns".format(df.shape[0], df.shape[1]))
#df.head()
print(df['message'][5])
print(df.loc[0]["message"])


Loaded 517401 emails with 2 columns
Message-ID: <30965995.1075863688265.JavaMail.evans@thyme>
Date: Thu, 31 Aug 2000 04:17:00 -0700 (PDT)
From: phillip.allen@enron.com
To: greg.piper@enron.com
Subject: Re: Hello
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Greg Piper
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

Greg,

 How about either next Tuesday or Thursday?

Phillip
Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is ou

In [109]:
subset = df.loc[150000:300000]

subset.head()

,file,message
150000,guzman-m/notes_inbox/1120.,"Message-ID: <7311614.1075840720380.JavaMail.evans@thyme>\nDate: Wed, 7 Feb 2001 17:12:00 -0800 (PST)\nFrom: ryan.slinger@enron.com\nTo: portland.shift@enron.com\nSubject: Just an Idea\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Ryan Sl..."
150001,guzman-m/notes_inbox/1121.,"Message-ID: <9976025.1075840720405.JavaMail.evans@thyme>\nDate: Wed, 7 Feb 2001 16:29:00 -0800 (PST)\nFrom: enron.announcements@enron.com\nTo: ena.employees@enron.com\nSubject: XMS (Expense Management System)\nMime-Version: 1.0\nContent-Type: text/plain; charset=ANSI_X3.4-1968\nContent-Transfer-..."
150002,guzman-m/notes_inbox/1122.,"Message-ID: <29112607.1075840720462.JavaMail.evans@thyme>\nDate: Wed, 7 Feb 2001 01:59:00 -0800 (PST)\nFrom: rick@osubeavers.com\nTo: knewlon@baughpdx.com, dylanand@home.com, matt_arnzen@emeraldsolutions.com, \n\tkris_barber@yahoo.com, tom.bauer@wrei.com, andrew.beattie@wfsg.com, \n\tbrian.hogue..."
150003,guzman-m/notes_inbox/1123.,"Message-ID: <4861278.1075840720485.JavaMail.evans@thyme>\nDate: Tue, 6 Feb 2001 08:40:00 -0800 (PST)\nFrom: cara.semperger@enron.com\nTo: portland.shift@enron.com\nSubject: Schedules going real time that need your attention\nMime-Version: 1.0\nContent-Type: text/plain; charset=ANSI_X3.4-1968\nCo..."
150004,guzman-m/notes_inbox/1124.,"Message-ID: <32027652.1075840720508.JavaMail.evans@thyme>\nDate: Tue, 6 Feb 2001 08:36:00 -0800 (PST)\nFrom: cara.semperger@enron.com\nTo: portland.desk@enron.com\nSubject: Ski Trip Photos and Video!\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\..."


In [111]:
def extract_messages(emails):
    messages = []
    for item in emails["message"]:
        # Return a message object structure from a string
        e = email.message_from_string(item)    
        # get message body  
        message_body = e.get_payload()
        messages.append(message_body)
    print("Successfully retrieved message body from e-mails!")
    return messages

bodies = extract_messages(subset)

Successfully retrieved message body from e-mails!


In [112]:
for index in bodies[:10]:

    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print(index)
    print("===============================================================")
    print("\n")

pd.set_option('display.max_colwidth', 300)
newBodiesDF = pd.DataFrame(bodies).rename(columns ={0:"Messages"})

newBodiesDF.head()


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
I realilzed we may be focusing a little too much on malin and leaving some 
rocks unturned.  I talked to Sempra and we can use them as a sleeve for 
different counterparties that we don't not have a credit agreement 
established with(i.e. metrowater @ mead and others).  They generally charge 
$2.

ryan.


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Effective March 1, 2001 Enron North America (CO 0413) employees will begin=
=20
using XMS (Expense Management System) to process employee expense=20
reimbursements.

XMS is a web based employee expense reimbursement system which will provide=
=20
the following benefits;

? More timely reimbursements to employees
? User friendly Web access
? Ability for employee or designated administrative assistant to submit=20
expense reimbursement
? Greater efficiency and lower cost
? Efficiently integrated with the current SAP accounting system

This system will REPLACE our curre

,Messages
0,I realilzed we may be focusing a little too much on malin and leaving some \nrocks unturned. I talked to Sempra and we can use them as a sleeve for \ndifferent counterparties that we don't not have a credit agreement \nestablished with(i.e. metrowater @ mead and others). They generally charge ...
1,"Effective March 1, 2001 Enron North America (CO 0413) employees will begin=\n=20\nusing XMS (Expense Management System) to process employee expense=20\nreimbursements.\n\nXMS is a web based employee expense reimbursement system which will provide=\n=20\nthe following benefits;\n\n? More timely r..."
2,_____________________________________________________________\nhttp://www.osubeavers.com - The Official Athletic Site of Oregon State\nBeaver Fans! Get your FREE email at --> http://mail.osubeavers.com!\n - jackson.jpg
3,"We have two schedules that are going real time, one has been tagged \nsucessfully and shouldn't need help, EPE-LDWP tag 16178. The other one had \nthe real time tag denied because they don't want it submitted so early \n(#!#$@#$%)??PNM-LDWP, DELAYED tag 16179. Please complete this tag before..."
4,"New photos are being uploaded every day, keep checking back for more!\n\nPaste this address into your browser window: \nP:\Scheduling\IntranetDev\Ski Trip Info.html"


In [113]:
model = SentenceTransformer('sentence-transformers/paraphrase-distilroberta-base-v1')

def preprocess_text(text: str):
    return re.sub(r'[^A-Za-z0-9]+', ' ', text.strip())

def embedding_fn(sentences):      
    return model.encode(sentences)

def recommend(query, top_n):
    query_embedd = embedding_fn(preprocess_text(query))
    return newBodiesDF.Embeddings.apply(
        lambda body: util.pytorch_cos_sim(query_embedd, body).detach().numpy().flatten()
    ).astype(float).nlargest(top_n)

In [114]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [115]:

newBodiesDF['Embeddings'] = newBodiesDF['Messages'].apply(lambda x: model.encode(x))

newBodiesDF

,Messages,Embeddings
0,I realilzed we may be focusing a little too much on malin and leaving some \nrocks unturned. I talked to Sempra and we can use them as a sleeve for \ndifferent counterparties that we don't not have a credit agreement \nestablished with(i.e. metrowater @ mead and others). They generally charge ...,"[-0.038347587, 0.12836666, -0.06287681, 0.40641582, 0.40387565, -0.55847776, 0.014949474, 0.11674233, -0.049743567, -0.20739216, 0.08720669, 0.48628837, 0.13502292, 0.1386941, 0.34409973, 0.3628969, -0.30011746, 0.5182001, 0.39234665, -0.17569518, 0.09125688, -0.2378243, 0.2552463, 0.33987457, 0..."
1,"Effective March 1, 2001 Enron North America (CO 0413) employees will begin=\n=20\nusing XMS (Expense Management System) to process employee expense=20\nreimbursements.\n\nXMS is a web based employee expense reimbursement system which will provide=\n=20\nthe following benefits;\n\n? More timely r...","[0.12049992, -0.013185244, 0.12002699, 0.0703534, 0.13500063, -0.14789836, -0.042122897, 0.14732689, -0.025667515, 0.02962503, 0.3787696, -0.34810397, 0.06260061, 0.009898062, 0.48703653, -0.58631074, -0.05003137, 0.016377535, 0.3083983, 0.54143643, 0.015983164, 0.094749495, -0.006171651, -0.171..."
2,_____________________________________________________________\nhttp://www.osubeavers.com - The Official Athletic Site of Oregon State\nBeaver Fans! Get your FREE email at --> http://mail.osubeavers.com!\n - jackson.jpg,"[-0.08640585, -0.09471666, -0.046008125, -0.6067845, -0.035403486, 0.23198606, -0.048584234, 0.08903772, 0.2752767, -0.21868858, -0.17990984, 0.3213123, 0.3916287, 0.05118384, 0.19351882, -0.120394975, -0.15767094, -0.16471958, 0.005531096, -0.18493713, 0.09347901, 0.042245127, 0.26079312, 0.091..."
3,"We have two schedules that are going real time, one has been tagged \nsucessfully and shouldn't need help, EPE-LDWP tag 16178. The other one had \nthe real time tag denied because they don't want it submitted so early \n(#!#$@#$%)??PNM-LDWP, DELAYED tag 16179. Please complete this tag before...","[-0.017191889, 0.098781474, 0.097408324, 0.44860247, 0.07783559, -0.28778797, -0.092402704, 0.33492616, 0.027734483, -0.20023054, 0.7280511, -0.65464115, -0.036510304, -0.17037712, 0.43718505, -0.44739065, -0.0221715, 0.29007274, 0.2260386, -0.14726125, -0.011647227, -0.35709733, 0.22834077, 0.1..."
4,"New photos are being uploaded every day, keep checking back for more!\n\nPaste this address into your browser window: \nP:\Scheduling\IntranetDev\Ski Trip Info.html","[0.037929267, -0.20581199, -0.10028383, 0.17286164, 0.43920097, -0.027110448, -0.2030448, -0.46853152, -0.30022034, 0.074018955, 0.0008830583, -0.7817909, -0.057085942, 0.27501482, -0.13678375, 0.19222309, 0.03944696, 0.10196333, 0.19586994, 0.2185738, 0.39169648, 0.121060446, 0.10350085, -0.158..."
...,...,...
149996,---------------------- Forwarded by Kay Mann/Corp/Enron on 02/22/2001 10:26=\n=20\nAM ---------------------------\n\n\nkathleen.carnahan@enron.com on 02/22/2001 10:23:42 AM\nPlease respond to kathleen.carnahan@enron.com\nTo: kay.mann@enron.com\ncc: =20\n\nSubject: Individual.com - News From a Fr...,"[-0.005001804, 0.14199866, 0.03267484, 0.16737682, 0.25973397, 0.23717028, -0.054374713, 0.26303002, 0.01042399, -0.24531406, -0.024460375, 0.4091328, -0.28064916, -0.4571641, 0.06380941, -0.46919036, -0.10885823, 0.088477775, 0.28249767, 0.085395426, 0.050297756, -0.3709281, 0.4233336, 0.280822..."
149997,"You may have seen this...\n---------------------- Forwarded by Kay Mann/Corp/Enron on 02/22/2001 09:24 \nAM ---------------------------\nFrom: Mark E Haedicke@ECT on 02/21/2001 03:17 PM\nSent by: Janette Elbertson@ECT\nTo: Alan Aronowitz/HOU/ECT@ECT, Roger Balog/HOU/ECT@ECT, Peggy \nBanczak/HOU/...","[0.031270523, 0.24564786, 0.045003206, 0.14748286, 0.1434615, 0.015147688, -0.32042122, 0.16842844, 0.16839275, -0.18026431, -0.2123372, 0.20939635, -0.17045432, -0.09259538, -0.09431979, -0.2553841, 0.05481836, 0.40722385, 0.2580802, 0.3976985

In [132]:
search_test = ['attachment', 'job']
search_test = ['please find attached']
#search_test = ['let s grab lucnh', 'eat', 'dinner']
search_test = ['out of office','vacation','holiday', 'visit']


query = '\n'.join(search_test)

results = recommend(query, 10)
print(results)

7166      0.523670
8375      0.523670
9119      0.523670
31240     0.518418
43599     0.518418
136980    0.496994
13736     0.496460
11619     0.495650
12254     0.495650
12760     0.495650
Name: Embeddings, dtype: float64


In [133]:
for key, val in results.items():
    print(f'Score: {"{:.4f}".format(val)}')
    print(f'Email Body: {newBodiesDF.loc[key]["Messages"]}')
    print(f"=======================================================================")

Score: 0.5237
Email Body: How much vacation/floating holiday time have I left, any?
Score: 0.5237
Email Body: How much vacation/floating holiday time have I left, any?
Score: 0.5237
Email Body: How much vacation/floating holiday time have I left, any?
Score: 0.5184
Email Body: FYI:

Brent Hendry will be out of the office on vacation Thursday and Friday, May 
31st and June 1st.
Score: 0.5184
Email Body: FYI:

Brent Hendry will be out of the office on vacation Thursday and Friday, May 
31st and June 1st.
Score: 0.4970
Email Body: I am on vacation and will return to the office on March 18. 
Score: 0.4965
Email Body: I know I haven't started my vacation (7/27 P.M. through 8/3) yet.

I need to take another vacation day on 8/8 (Wednesday) to go to my property tax protest hearing.
Score: 0.4957
Email Body: In order to use up  my remaining vacation for the year, I will be out the 
following dates:

Friday 12/15
Friday 12/22
Christmas week 12/25 - 12/29

I will be back in the office on Tuesday 